In [32]:
# Импортируем библиотеки
import pandas as pd
import datetime

In [33]:
# Считываем данные из файла
df = pd.read_excel('data/data.xlsx')

In [34]:
# Отсортируем данные по именам и возрастании даты
df = df.sort_values(by=['merchant_name', 'day'])

In [35]:
# Удаляем месяц, так как он не несет никакой информации
df = df.drop('month', axis=1)

# Удалим строки с merchant_name, которое встречается меньше 6 раз в df
df = df.groupby('merchant_name').filter(lambda x : len(x) > 5)

In [36]:
# Получает на вход дату и возвращает день недели в виде его номера
def get_weekday(date):
    date = datetime.date(*map(int, str(str(date).split()[0]).split('-')))
    return date.weekday()

# Функция принимает на вход 2 даты и возвращает разницу между ними в днях
def get_days_diff(date1, date2):
    date1 = datetime.date(*map(int, str(str(date1).split()[0]).split('-')))
    date2 = datetime.date(*map(int, str(str(date2).split()[0]).split('-')))
    return (date2 - date1).days

# Функция принимает на вход дату и возвращает следующий день
def get_next_day(date):
    date = datetime.date(*map(int, str(str(date).split()[0]).split('-')))
    return str(date + datetime.timedelta(days=1))

# Введем новые параметры
df['weekday'] = df['day'].apply(get_weekday) 
df['is_weekend'] = df['weekday'].apply(lambda x : 1 if x in [5, 6] else 0)

In [37]:
# Если между двумя строками разница 2 дня, то добавим строку между ними с пропущенными значениями
for i in range(len(df) - 1):
    if get_days_diff(df.iloc[i]['day'], df.iloc[i + 1]['day']) == 2:
        df = df._append({
            'merchant_name' : df.iloc[i]['merchant_name'], 
            'day' : get_next_day(df.iloc[i]['day']),
            'cashback' : (df.iloc[i]['cashback'] + df.iloc[i + 1]['cashback']) / 2, 
            'weekday' : get_weekday(get_next_day(df.iloc[i]['day'])),
            'is_weekend' : 1 if get_weekday(get_next_day(df.iloc[i]['day'])) in [5, 6] else 0},
            ignore_index=True)

# Отсортируем данные по именам и возрастании даты
df = df.sort_values(by=['merchant_name', 'day'])

In [38]:
# Для каждого значения merchant_name построим график cashback
'''
for merchant_name in df['merchant_name'].unique():
    sns.lineplot(data=df[df['merchant_name'] == merchant_name], x='day', y='cashback')
    plt.title(merchant_name)
    plt.savefig(f'graphs/{merchant_name.replace('*', ' ')}.png')
    plt.close()
'''

"\nfor merchant_name in df['merchant_name'].unique():\n    sns.lineplot(data=df[df['merchant_name'] == merchant_name], x='day', y='cashback')\n    plt.title(merchant_name)\n    plt.savefig(f'graphs/{merchant_name.replace('*', ' ')}.png')\n    plt.close()\n"

In [39]:
# Сохраним df как clear_data.xlsx  

df.to_excel('data/clear_data.xlsx', index=False) 
